In [120]:
import numpy as np
import random
neighborlista=np.array([[0,0,0,1],[0,0,-1,1],[-1,0,0,1],[-1,0,-1,1],[0,-1,0,1],[0,-1,-1,1]])
neighborlistb=-neighborlista
def find_neighbor(particle):
    if particle[3]==1:
        return neighborlista+np.tile(particle,(6,1))
    if particle[3]==2:
        return neighborlistb+np.tile(particle,(6,1))
def bond_corr(bond_array_one):

    return np.array([bond_array_one[0]|bond_array_one[1],bond_array_one[0]|bond_array_one[1],bond_array_one[2]|bond_array_one[3],bond_array_one[2]|bond_array_one[3],bond_array_one[4]|bond_array_one[5],bond_array_one[4]|bond_array_one[5]])


class Simulation:
    def __init__(self):
        self.particle_array = np.empty((0, 4), dtype=int)
        self.neighbor_array = np.empty((0, 4), dtype=int)
        self.multiplicity_array = np.empty((0, 1), dtype=int)
        self.bond_number_array = np.empty((0, 1), dtype=int)
        self.bond_array=np.empty((0, 6), dtype=int)
        self.default_bond_array=np.array([0,0,0,0,0,0])
        self.bond_array=np.vstack((self.bond_array,self.default_bond_array)) 

        self.events = []
        self.time = 0
        self.g_0=0
        self.g_1=0
        self.free_bonds_energy=0
    def add_particle(self, i):# i is the index of the added particle in the original neighbor array and multiplicity array
        #add the particle from neighbor array to particle array, update the neighbor array, multiplicity array and bond array accordingly
        particle=self.neighbor_array[i]
        self.bond_array=np.vstack((self.bond_array,self.default_bond_array)) 
        if self.bond_number_array.shape[0]!=0:
            self.bond_number_array=np.vstack((self.bond_number_array,self.multiplicity_array[i]))   
            
        else:
            self.bond_number_array=np.vstack((self.bond_number_array,np.array([0])))
            self.bond_number_array=np.vstack((self.bond_number_array,self.multiplicity_array[i]))  
        self.neighbor_array=np.delete(self.neighbor_array,i,axis=0)
        self.multiplicity_array=np.delete(self.multiplicity_array,i,axis=0)
        self.particle_array = np.vstack((self.particle_array, particle))
        index_added=self.particle_array.shape[0]-1
        neighbors=find_neighbor(particle)
        for j in range(0,6):
            if np.where((self.particle_array == tuple(neighbors[j].tolist())).all(axis=1))[0].size==1:
                index=np.where((self.particle_array == tuple(neighbors[j].tolist())).all(axis=1))[0][0]
                self.bond_number_array[index][0]+=1
                self.bond_array[index][j]+=1
                self.bond_array[index_added][j]+=1
            if np.where((self.neighbor_array == tuple(neighbors[j].tolist())).all(axis=1))[0].size==1:
                self.multiplicity_array[np.where((self.neighbor_array == tuple(neighbors[j].tolist())).all(axis=1))[0][0]][0]+=1
            else:
                self.neighbor_array=np.vstack((self.neighbor_array,neighbors[j]))
                self.multiplicity_array=np.vstack((self.multiplicity_array,np.array([1])))
    def remove_particle(self,i):#i is the index of the removed particle in the orginal particle array, bond number array and bond array
        #remove the particle from particle array, update the neighbor array, multiplicity array and bond array accordingly
    
        self.neighbor_array=np.vstack((self.neighbor_array,self.particle_array[i]))
        self.multiplicity_array=np.vstack((self.multiplicity_array,self.bond_number_array[i]))
        vacancy=self.particle_array[i]
        neighbors=find_neighbor(vacancy)
        self.particle_array=np.delete(self.particle_array,i,axis=0)
        self.bond_number_array=np.delete(self.bond_number_array,i,axis=0)
        self.bond_array=np.delete(self.bond_array,i,axis=0)
        for j in range(0,6):
            if np.where((self.particle_array == tuple(neighbors[j].tolist())).all(axis=1))[0].size==1:
                index=np.where((self.particle_array == tuple(neighbors[j].tolist())).all(axis=1))[0][0]
                self.bond_number_array[index][0]-=1
                
                if self.bond_number_array[index][0]==0:
                    self.remove_particle(index)
                else:
                    self.bond_array[index][j]-=1

            if np.where((self.neighbor_array == tuple(neighbors[j].tolist())).all(axis=1))[0].size==1:
                index=np.where((self.neighbor_array == tuple(neighbors[j].tolist())).all(axis=1))[0][0]
                self.multiplicity_array[index][0]-=1
                if self.multiplicity_array[index][0]==0:
                    self.neighbor_array=np.delete(self.neighbor_array,index,axis=0)
                    self.multiplicity_array=np.delete(self.multiplicity_array,index,axis=0) 
        

    def insert_one_particle_event(self, i):#i is the index of the proposed inserted particle in the original neighbor list and multiplicity list
        # ... Define the event, with a rate based on the multiplicity
        rate = 1 / (4 ** self.multiplicity_array[i][0])
        self.events.append(('insert', i, rate))

    def delete_one_particle_event(self, i):#i is the index of the proposed removed particle in the original particle list, bond number list and bond list
        if self.bond_number_array.shape[0]!=0:
            rate = self.calculate_delete_rate(i)
            self.events.append(('delete', i, rate))
       

    def calculate_delete_rate(self, i):#i is the index of the proposed removed particle in the original particle list
        bond_number=self.bond_number_array[i][0]
        particle=self.particle_array[i]
        neighbors=find_neighbor(particle)
        free_bonds=0
        self_bond_state=bond_corr(self.bond_array[i])
        free_bonds+=self_bond_state.sum()
        for j in range(0,6):
            if np.where((self.particle_array == tuple(neighbors[j].tolist())).all(axis=1))[0].size==1:
                index=np.where((self.particle_array == tuple(neighbors[j].tolist())).all(axis=1))[0][0]
                neighbor_bond=self.bond_array[index].copy()
                old_neighbor_bond_state=bond_corr(neighbor_bond)
                new_neighbor_bond=neighbor_bond.copy()
                new_neighbor_bond[j]=0
                new_neighbor_bond_state=bond_corr(new_neighbor_bond)
                bond_change=new_neighbor_bond_state.sum()-old_neighbor_bond_state.sum()
                free_bonds-=bond_change


            
        g=self.g_0+self.bond_number_array[i][0]*self.g_1-free_bonds*self.free_bonds_energy
        return np.exp(-g) / (4 ** self.bond_number_array[i][0])
        
    
    def execute_event(self, event):
        if event[0]=='insert':
            self.add_particle(event[1])
            print("insert")
        if event[0]=='delete':
            self.remove_particle(event[1])
            print("remove")

    def kmc_step(self):
        self.events=[]
        for i in range(0,self.neighbor_array.shape[0]):
            self.insert_one_particle_event(i)
        for i in range(0,self.particle_array.shape[0]):
            self.delete_one_particle_event(i)
        total_rate = sum(event[2] for event in self.events)
        rand_number=random.random()
        dt = -np.log(rand_number) / total_rate
        self.time += dt
        cumulative_rate = 0
        r = rand_number * total_rate
        for event in self.events:
            cumulative_rate += event[2]
            if r < cumulative_rate:
                self.execute_event(event)
                break

    def run(self, steps):
        for _ in range(steps):
            self.kmc_step()

# Create the simulation object
sim = Simulation()

# Add the initial particle
sim.particle_array=np.array([0,0,0,1])
sim.neighbor_array=find_neighbor(np.array([0,0,0,1]))
sim.multiplicity_array=np.array([[1],[1],[1],[1],[1],[1]])
sim.g_0=-5
sim.g_1=10
sim.free_bonds_energy=1
# Run the simulation for a number of steps
sim.run(100)


insert
insert
insert
insert
insert
remove
insert
insert
insert
remove
insert
remove
insert
insert
insert
insert
remove
insert
insert
insert
insert
remove
insert
insert
insert
insert
insert
remove
insert
remove
insert
insert
insert
insert
insert
remove
insert
insert
insert
insert
remove
insert
insert
insert
remove
insert
insert
remove
insert
remove
remove
remove
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
remove
insert
remove
insert
insert
insert
insert
insert
insert
insert
insert
insert
insert
remove
insert
insert
insert
insert
remove
insert
remove
insert
insert
insert
insert
insert
insert
insert
insert
remove
insert
remove


In [93]:
bond_corr(sim.bond_array[0])

array([0, 0, 0, 0, 0, 0])

In [100]:
sim.particle_array.shape[0]

3

In [126]:
sim.multiplicity_array

array([[3],
       [5],
       [4],
       [3],
       [2],
       [3],
       [3],
       [1],
       [2],
       [4],
       [1],
       [2],
       [4],
       [5],
       [3],
       [5],
       [3],
       [5],
       [4],
       [3],
       [2],
       [2],
       [1],
       [2],
       [4],
       [1],
       [3],
       [1],
       [1],
       [2],
       [3],
       [2],
       [1],
       [1],
       [1],
       [4],
       [3],
       [3],
       [4],
       [3],
       [2],
       [2],
       [2],
       [3],
       [1],
       [4],
       [2],
       [1],
       [2],
       [1],
       [1],
       [2],
       [1],
       [2],
       [2],
       [2],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [3],
       [1],
       [1],
       [1],
       [1],
       [2],
       [3],
       [1],
       [1],
       [2],
       [2],
       [2],
    

In [125]:
sim.neighbor_array

array([[ 0,  0,  0,  2],
       [-1,  0,  0,  2],
       [-1,  0, -1,  2],
       [ 0,  0,  0,  1],
       [ 0, -1, -1,  1],
       [ 1, -1,  0,  1],
       [ 0, -1, -1,  2],
       [ 0, -2, -1,  2],
       [ 0,  1, -1,  2],
       [-1,  1, -1,  2],
       [ 0,  1,  1,  1],
       [-1, -1,  0,  1],
       [ 0, -1,  0,  1],
       [ 0, -1,  1,  1],
       [-1,  0,  1,  1],
       [ 0,  0, -1,  2],
       [-1,  1,  1,  2],
       [-1,  1,  0,  2],
       [-2,  1,  1,  2],
       [ 0,  0,  1,  1],
       [ 0,  0,  2,  1],
       [-1,  1,  2,  1],
       [-1,  0,  2,  2],
       [-1,  0,  1,  2],
       [-2,  0,  2,  2],
       [-1, -1,  1,  2],
       [-2,  0,  0,  2],
       [-2,  0, -1,  2],
       [ 0, -2,  0,  1],
       [ 1, -2,  0,  1],
       [ 1, -2,  1,  1],
       [-1,  2,  0,  2],
       [-1,  2, -1,  2],
       [-1,  2,  1,  2],
       [-1,  0,  2,  1],
       [ 1, -1, -1,  2],
       [ 1, -1, -2,  2],
       [ 0, -1, -2,  2],
       [-2,  2,  0,  1],
       [-2,  2,  1,  1],


In [123]:
sim.bond_number_array

array([[3],
       [6],
       [4],
       [5],
       [2],
       [6],
       [2],
       [3],
       [3],
       [2],
       [2],
       [3],
       [3],
       [2],
       [3],
       [5],
       [3],
       [3],
       [2],
       [4],
       [1],
       [3],
       [4],
       [2],
       [1],
       [3],
       [2],
       [2],
       [3],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [4],
       [2],
       [1],
       [1],
       [3],
       [1],
       [1],
       [2],
       [2],
       [1],
       [1],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [1],
       [2],
       [1],
       [1],
       [2]])

In [124]:
sim.bond_array

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [0, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 0],
       [0, 0, 1, 0, 0, 1],
       [1, 2, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 1, 0, 0, 2, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 1],
       [1, 0, 0, 1, 1, 0],
       [1, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1],
       [0, 0, 1, 1, 0, 1],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1],
       [0, 1, 1, 2, 0, 0],
       [0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1],
 

In [118]:
sim.multiplicity_array

array([[ 1],
       [ 1],
       [-1]])

In [49]:
bond_array_one=np.array([1,1,1,1,1,1])
bond_array_one_logic=np.ones((6), dtype=bool)

for i in range(0,6):
    if bond_array_one[i]==-1:
        bond_array_one_logic[i]=False
    else:
        bond_array_one_logic[i]=True
np.array([bond_array_one_logic[0]|bond_array_one_logic[1],bond_array_one_logic[0]|bond_array_one_logic[1],bond_array_one_logic[2]|bond_array_one_logic[3],bond_array_one_logic[2]|bond_array_one_logic[3],bond_array_one_logic[4]|bond_array_one_logic[5],bond_array_one_logic[4]|bond_array_one_logic[5]])


array([ True,  True,  True,  True,  True,  True])

In [47]:
bond_array_one_logic

array([1., 1., 1., 1., 1., 1.])

In [121]:
position_array=np.zeros((sim.particle_array.shape[0],3))
atom_array=np.array([[0,0,0],[1.73,1.73/np.sqrt(3),0.94]])
lattice_array=np.array([[3.46,0,0],[3.46/2,3.46*np.sqrt(3)/2,0],[0,0,1.88]])
for i in range(0,sim.particle_array.shape[0]):
    position_array[i]=sim.particle_array[i][0]*lattice_array[0]+sim.particle_array[i][1]*lattice_array[1]+sim.particle_array[i][2]*lattice_array[2]+atom_array[sim.particle_array[i][3]-1]

In [122]:
import plotly.graph_objects as go
import numpy as np

# Helix equation

x, y, z = position_array[0:300,0],position_array[0:300,1],position_array[0:300,2]

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers')])
fig.show()

In [112]:
sim.particle_array

array([[  0,   0,   0,   1],
       [  0,   0,   0,   3],
       [  0,   0,   0,   2],
       ...,
       [ -1,  17,  -7,   3],
       [-14,   9,  -4,   4],
       [  0,   4,   0,   2]])

In [22]:
neighbor_array.shape

(1, 4)

In [21]:
for i in neighbor_array:
    print(i)

[4 0 0 0]


In [23]:
neighbor_array[-1]

array([4, 0, 0, 0])

In [26]:
np.empty((0, 1), dtype=int)

array([], shape=(0, 1), dtype=int64)

In [27]:
np.full(
  shape=6,
  fill_value=1).shape

(6,)

In [31]:
a=np.array([[1],[1],[1],[1],[1],[1]])

In [51]:
np.vstack((a,np.array([1])))

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [48]:
a

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]])